In [ ]:
# Notes

# Environment.yml for deploy
# Create tiffs from cubes and clip to overlap using the warp/translate
# Plot axes/label axes
# Function documentation
# Bin script tiff creation
# Analysis bin script
    # - Quiver png
    # - Homography.txt (9 numbers)
    # - Dataframe to csv (keep names the same)
    # - Stats to csv (Dataframe describe data + quantile data(x, y, magnitude))

# Box to Histogram
# Correlation/Coregistration in Z dimension
    # - Apply homography
    # - Difference the two DTMs for some Z offset

#-------Nice To Have-------
# Specify meters or pixels for plotting

# Be sure to switch the kernel to autocnet

In [ ]:
import os
import sys
import cv2
import json
import gdal
import math
import affine
from osgeo import ogr

import numpy as np
import scipy
import pandas as pd
import cv2

from plio.geofuncs import geofuncs
from plio.io.io_gdal import GeoDataset
from autocnet.matcher import subpixel as sp
from autocnet.transformation import homography as hg
from autocnet.camera import camera
from autocnet.transformation.fundamental_matrix import compute_fundamental_matrix

import matplotlib.pyplot as plt
%matplotlib inline

plt.rcParams['image.cmap'] = 'plasma'

# Paths and Files

Similarly to the Image Conversion notebook this notebook requires a similar image setup. For now, I would copy the contents of that cell of the image conversion notebook into the cell bellow.
Each cube requires a "basepath" and the cubes name.

The basepath is the path to the directory containing the cubes. Ideally, you would want to place mutiple cubes within the same directory
to make opening and accessing them easier. In the cell bellow, define various basepaths and cub names.

You then can define the two cubes however you want using the basepath and a cube.

In [ ]:
# setup the paths to cubes and tiffs
hirise_basepath = '/home/acpaquette/repos/mars2020_trn/TestData/HiRISE_Jezero/'
ctx_20_basepath = '/home/acpaquette/repos/mars2020_trn/TestData/CTX_Jezero/20m_ctx/'
ctx_6_basepath = '/home/acpaquette/repos/mars2020_trn/TestData/CTX_Jezero/6m_ctx/'
hrsc_basepath = '/home/acpaquette/repos/mars2020_trn/TestData/HRSC_Jezero/'

hirise_dem1 = os.path.join(hirise_basepath, 'DEM_1m_Jezero_CE_isis3.cub')
hirise_dem2 = os.path.join(hirise_basepath, 'DEM_1m_Jezero_C_isis3.cub')

ctx_dem1 = os.path.join(ctx_6_basepath, 'tfm_abso_Jezero_F05_V6_IAUsph_adj_XYZposAndVelAndAngles_20m_onePassAfterngate.tiff')
ctx_dem2 = os.path.join(ctx_6_basepath, 'tfm_abso_Jezero_J03_V6_IAUsph_adj_XYZposAndVelAndAngles_20m_onePassAfterngate.tiff')

# setup the paths to cubes and tiffs
hirise_cub1 = "ESP_023524_1985_1m_o_isis3.cub"
hirise_cub2 = "ESP_048908_1985_1m_o_isis3.cub"
ctx_cub1 = "F05_037607_2008_XN_20N282W_v6_PosAndVelAndAngles_20m_o.cub"
ctx_cub2 = "J03_045994_1986_XN_18N282W_v6_20m_o.cub"
ctx_cub3 = "F05_037607_2008_XN_20N282W_v6_PosAndVelAndAngles_6m_o.cub"
ctx_cub4 = "J03_045994_1986_XN_18N282W_v6_6m_o.cub"
hrsc_cub = "H5270_0000_ND4.IMG"

# cub_image1 = os.path.join(hirise_basepath, hirise_cub1)
# cub_image2 = os.path.join(hirise_basepath, hirise_cub2)

# cub_image1 = os.path.join(ctx_6_basepath, ctx_cub3)
# cub_image2 = os.path.join(ctx_6_basepath, ctx_cub4)

cub_image1 = os.path.join(ctx_20_basepath, ctx_cub1)
cub_image2 = os.path.join(ctx_20_basepath, ctx_cub2)

# cub_image1 = hirise_dem1
# cub_image2 = hirise_dem2

# cub_image1 = ctx_dem1
# cub_image2 = ctx_dem2

# Variables
You can also define

* Search Size 
    * Some integer i to generate and i x i search space
    * Increasing this value will make the overall process take longer but be more precise
* Template 
    * Size Some integer i to generate and i x i template
    * Increasing this value will make the overall process shorter but less precise
* Grid Size 
    * Some integer i to generate an i by j grid of points, where j is determined dynamically
    * Increasing this value will make the overall process longer as there will be more points on the image
* Correlation Threshold 
    * Some float <= 1 for how corrlated two points need to be to include in homography calculation
    * Increasing this value will limit the system to points that are within the given percentile
* Use Default Graph Values 
    * As long as this is True, it will autoscale the CDF graphs, if it is turned to False it will then use the   values defined after it to set the grid range 
* X Axis Values 
    * x_start: The x axis starting point for the range of values desired for preview on the CDF graph
    * x_end: The x axis end point for the range of values desired for preview on the CDF graph
    * x_step: The value of each step that will be taken between (x_start, x_end) Ex. A step value of 5 where the range of values is (0, 100) this would produce 20 ticks on the x axis.
* Y Axis Values
    * y_start: The y axis starting point for the range of values desired for preview on the CDF graph
    * y_end: The y axis end point for the range of values desired for preview on the CDF graph
    * y_step: The value of each step that will be taken between (y_start, y_end) Ex. A step value of 5 where the range of values is (0, 100) this would produce 20 ticks on the y axis.

In [ ]:
search_size = 101
template_size = 25
grid_size = 20
corr_threshold = .75

use_default_graph_values = True

# Y Axis Values
x_start = 0
x_end = 100
x_step = 1

# X Axis Values
y_start = 0
y_end = 100
y_step = 1

# Running the rest of it

Once you have defined the cubes you want and the variables run the rest of the cells and wait for output

In [ ]:
# Creates a pandas dataframe of the image information you are currently using
def image_summary(tiff_geo1, tiff_geo2):
    
    # Allows dataframe to show entire result
    pd.set_option('display.max_colwidth', -1)
    tiff_image1 = os.path.splitext(cub_image1)[0] + '.tiff'
    tiff_image2 = os.path.splitext(cub_image2)[0] + '.tiff'
    image_data = 'cub_image1', 'cub_image2'
    columns = {'File Type': [tiff_image1.split('.')[-1], tiff_image2.split('.')[-1]],
               'File Path': [tiff_image1, tiff_image2],
               'Pixel Resolution': [tiff_geo1.pixel_width, tiff_geo2.pixel_width], 
               'Units': [tiff_geo1.scale[0], tiff_geo2.scale[0]],
               'Image Name': [tiff_geo1, tiff_geo2]}
    
    df = pd.DataFrame(index=image_data, data=columns)
    
    # Reorder the dataframe columnds to th e right order
    df = df[['Image Name', 'Pixel Resolution', 'Units','File Type', 'File Path']]
    
    return df

# Plot the images and there overlapping grid area
def show_coregistration(source_image, destination_image, **kwargs):
    plt.figure(0, figsize=(10, 10))
    plt.imshow(source_image, **kwargs)
    plt.imshow(destination_image, **kwargs)
    plt.show()

# Show the quiver plot of the offsets
def display_quiver(comp_df, source_image, mask = [], scale = 100, scale_units = 'inches', **kwargs):
    if len(mask) != 0:
        comp_df = comp_df[mask]
    print(comp_df.columns)
    plt.figure(4, figsize=(20, 20))
    plt.imshow(source_image, cmap="Greys")
    plt.quiver(comp_df['destination_x'], comp_df['destination_y'], 
               -(comp_df['xoff']), (comp_df['yoff']),
               color = 'Red', scale = scale, scale_units = scale_units, **kwargs)
    plt.show()
    
# Given an index in the dataframe examine the before and after
# when the offset is applied
def examine_point(idx, size, comp_df, source_image, destination_image, mask = []):
    if len(mask) != 0:
        comp_df = comp_df[mask]
        
    plt.figure(2, figsize=(5, 5))
    plt.text(20, 50, 'Before Offset Correction', fontsize=12)
    x, y = int(comp_df.iloc[idx]['source_x']), int(comp_df.iloc[idx]['source_y'])
    plt.imshow(source_image[y - size:y + size, x - size:x + size], alpha = .5, cmap='Greys')

    x, y = int(comp_df.iloc[idx]['destination_x']), int(comp_df.iloc[idx]['destination_y'])
    plt.imshow(destination_image[y - size:y + size, x - size:x + size], alpha = .5, cmap='Greys')

    plt.figure(3, figsize=(5, 5))
    plt.text(20, 50, 'After Offset Correction', fontsize=12)
    x, y = int(comp_df.iloc[idx]['offset_source_x']), int(comp_df.iloc[idx]['offset_source_y'])
    plt.imshow(source_image[y - size: y + size, x - size: x + size], alpha = .5, cmap='Greys')

    x, y = int(comp_df.iloc[idx]['destination_x']), int(comp_df.iloc[idx]['destination_y'])
    plt.imshow(destination_image[y - size:y + size, x - size:x + size], alpha = .5, cmap='Greys')
    plt.show()
    
    offset_x, offset_y, corr = comp_df.iloc[idx][['xoff', 'yoff', 'corr']]
    print('X Offset: {}\nY Offset: {}\nCorrelation: {}'.format(offset_x, offset_y, corr))
    
def compute_homography(comp_df):
    x1 = np.array([*zip(comp_df['offset_source_x'].__array__(), comp_df['offset_source_y'].__array__())])
    x2 = np.array([*zip(comp_df['destination_x'].__array__(), comp_df['destination_y'].__array__())])
    H, mask = hg.compute_homography(x1, x2)
    
    return H, mask

# Apply the homography to the source image and display
def apply_homography(comp_df, image, H, height, width):
    result = cv2.warpPerspective(image, H, (height, width))
    
    img_min = np.nanmin(image)
    img_max = np.nanmax(image)
    
    result[result > img_max] = np.NAN
    result[result < img_min] = np.NAN
    
    return result
    
def generate_point_grid(source_geo, destination_geo, source_raster, destination_raster, size):
    # Compute the overlap and get the corners now that
    # we have the geometry
    overlap_hull = source_geo.compute_overlap(destination_geo)[0]

    # Get the lats and lons of the assocaited corners
    overlap_lon = [i[0] for i in overlap_hull]
    overlap_lat = [i[1] for i in overlap_hull]

    # Define a ratio so the distrabution is even
    overlap_ratio = (max(overlap_lon) - min(overlap_lon)) / (max(overlap_lat) - min(overlap_lat))

    lon = np.linspace(min(overlap_lon) + .001, max(overlap_lon) - .001, size)
    lat = np.linspace(min(overlap_lat) + .001, max(overlap_lat) - .001, round(size/overlap_ratio))

    # Get the lat, lon position for the grid
    lonv, latv = np.meshgrid(lon, lat, sparse=True)
    print('Generating a', len(lonv[0]), 'by', len(latv), 'point grid')

    coords = []

    # Begin looping over each point in the grid
    for lat_val in latv:
        for lon_val in lonv[0]:
            # Find the point in pixel space for each image and get the value
            x1, y1  = source_geo.latlon_to_pixel(lat_val[0], lon_val)
            x2, y2  = destination_geo.latlon_to_pixel(lat_val[0], lon_val)
            point_val1 = source_raster[y1 - 1, x1 - 1]
            point_val2 = destination_raster[y2 - 1, x2 - 1]

            # If either is zero then the point should be ignored
            # as it lies outside of the true overlap
            if point_val1 > 0 and point_val2 > 0:
                coords.append([x1, y1, x2, y2, lat_val[0], lon_val])

    # Build dataframe after grid contruction for data storage and 
    # ease of access
    df = pd.DataFrame(coords, columns = ['source_x', 
                                         'source_y', 
                                         'destination_x', 
                                         'destination_y', 
                                         'lat', 
                                         'lon'])
    return df

# The Meat and Potatoes of offset calculation
def compute_offsets(df, s_img, d_img, template_size, search_size, corr_threshold=0.9):
    offsets = []

    # Iterate through each point in the dataframe and calculate offsets
    print('Computing Offsets')
    for idx, row in df.iterrows():

        x, y = row['source_x'], row['source_y']
        s_template = sp.clip_roi(s_img, (x, y), template_size)


        x, y = row['destination_x'], row['destination_y']
        d_search = sp.clip_roi(d_img, (x, y), search_size)

        xoff, yoff, corr = sp.subpixel_offset(s_template, d_search)
        mag = np.linalg.norm([xoff, yoff])
        # Apply the offsets to the source points and 
        # save those as well
        offset_source_x = row['source_x'] - xoff
        offset_source_y = row['source_y'] + yoff
        offsets.append([offset_source_x, offset_source_y, xoff, yoff, mag, corr])
        sys.stdout.write('%s%s\r' % (round((idx/len(df) * 100)), '% complete'))
        sys.stdout.flush()
        
    off_df = pd.DataFrame(offsets, columns = ['offset_source_x', 'offset_source_y', 'xoff', 'yoff', 'mag', 'corr'])
    comp_df = df.merge(off_df, left_index=True, right_index=True)
    corr_mask = comp_df['corr'] > corr_threshold
    
    H, mask = compute_homography(comp_df[corr_mask])
    mask_index = comp_df[corr_mask][mask].index.__array__()
    full_mask = [True if i in mask_index else False for i, val in comp_df.iterrows()]
    return comp_df, H, full_mask

# Function to create CDF graphs for offsets
def calculate_cdf_graphs(comp_df, df_mask=True):
        
    # Set min/maxes for each column
    mag_max, mag_min = np.max(comp_df[mask]['mag']), np.min(comp_df[mask]['mag'])
    xoff_max, xoff_min = np.max(comp_df[mask]['xoff']), np.min(comp_df[mask]['xoff'])
    yoff_max, yoff_min = np.max(comp_df[mask]['yoff']), np.min(comp_df[mask]['yoff'])
    
    if not df_mask:
        # Set min/maxes for each column
        mag_max, mag_min = np.max(comp_df['mag']), np.min(comp_df['mag'])
        xoff_max, xoff_min = np.max(comp_df['xoff']), np.min(comp_df['xoff'])
        yoff_max, yoff_min = np.max(comp_df['yoff']), np.min(comp_df['yoff'])
        
    
    # Get cumulative data and the count
    mag_bins, mag_count, mag_cumulative = cumulative_stats(comp_df, column_name='mag', bin_start=mag_min, bin_end=mag_max)
    xoff_bins, xoff_count, xoff_cumulative = cumulative_stats(comp_df, column_name='xoff', bin_start=xoff_min, bin_end=xoff_max)
    yoff_bins, yoff_count, yoff_cumulative = cumulative_stats(comp_df, column_name='yoff', bin_start=yoff_min, bin_end=yoff_max)
    
    # Use the min and max to determine the base for each column
    mag_base = np.arange(mag_min, mag_max+1, 1)
    xoff_base = np.arange(xoff_min, xoff_max+1, 1)
    yoff_base = np.arange(yoff_min, yoff_max+1, 1)
    
    # Plot the cumulative function
    plt.figure(figsize=(15,15))
    mag = plt.plot(mag_base, mag_cumulative*100, label= "Magnitude", c='blue')
    xoff = plt.plot(xoff_base, xoff_cumulative*100, label= "X Offset", c='red')
    yoff = plt.plot(yoff_base, yoff_cumulative*100, label= "Y Offset", c='green')
    plt.legend(loc=5, fontsize="x-large")
    plt.grid(True)

    # Setup Labels for graph
    plt.xlabel('Pixel Offset Value', fontsize=15)
    plt.ylabel('Percentage of Values', fontsize=15)
    plt.title('CDF Mars2020 Pixel Offset', fontsize=18)
    if not use_default_graph_values:
        plt.xticks(np.arange(x_start, x_end, x_step))
        plt.yticks(np.arange(y_start, y_end, y_step))

    plt.show()
    
def cumulative_stats(comp_df, column_name='mag', bin_start=0, bin_end=100, bin_step=1, df_mask=True):
    
    # Grabs the masked data
    data = comp_df[mask][column_name]
    
    # Allows for using non-mask data
    if not df_mask:
        data = comp_df[column_name]
    
    # Generates the bins for cumulative stats
    bins = np.arange(bin_start, bin_end, bin_step)
    
    # Puts the data into bins
    organized_data = np.digitize(data, bins)
    
    # Counts the number in each bin
    count = np.bincount(organized_data)
    
    # Calculates total_count for percentage 
    total_count = np.sum(count)
    count_percentage = count / total_count
    
    # Calculates the cumulative sum using the count_percentage array
    cumulative_sum = np.cumsum(count_percentage)
    
    return bins, count, cumulative_sum

def calculate_cumulative_statistics_df(comp_df):

    # Setup values for min and max
    mag_max, mag_min = np.amax(comp_df[mask]['mag']), np.amin(comp_df[mask]['mag'])
    xoff_max, xoff_min = np.amax(comp_df[mask]['xoff']), np.amin(comp_df[mask]['xoff'])
    yoff_max, yoff_min = np.amax(comp_df[mask]['yoff']), np.amin(comp_df[mask]['yoff'])

    # Magnitude Cumulative Statistics
    mag_bins, mag_count, mag_cumulative_sum = cumulative_stats(comp_df, column_name='mag', bin_start=mag_min, bin_end=mag_max, bin_step=0.25)
    mag_bins = np.append(mag_bins, mag_max)

    # Xoff Cumulative Statistics
    xoff_bins, xoff_count, xoff_cumulative_sum = cumulative_stats(comp_df, column_name='xoff', bin_start=xoff_min, bin_end=xoff_max, bin_step=0.25)
    xoff_bins = np.append(xoff_bins, xoff_max)

    # Yoff Cumulative Statistics
    yoff_bins, yoff_count, yoff_cumulative_sum = cumulative_stats(comp_df, column_name='yoff', bin_start=yoff_min, bin_end=yoff_max, bin_step=0.25)
    yoff_bins = np.append(yoff_bins, yoff_max) 

    # Generates the data for the DataFrame
    mag_columns = {'Mag Count': mag_count, 'Mag Cumulative': mag_cumulative_sum, 'Mag Bins': mag_bins}
    xoff_columns = {'Xoff Count': xoff_count, 'Xoff Cumulative': xoff_cumulative_sum, 'Xoff Bins': xoff_bins}
    yoff_columns = {'Yoff Count': yoff_count, 'Yoff Cumulative': yoff_cumulative_sum, 'Yoff Bins': yoff_bins}

    # Generates the dataframe
    mag_df = pd.DataFrame(data=mag_columns)
    yoff_df = pd.DataFrame(data=yoff_columns)
    xoff_df = pd.DataFrame(data=xoff_columns)

    cum_df = pd.concat([xoff_df, yoff_df, mag_df], axis=1)
    
    return cum_df

In [ ]:
tiff_image1 = os.path.splitext(cub_image1)[0] + '.tiff'
tiff_image2 = os.path.splitext(cub_image2)[0] + '.tiff'

tiff_geo1 = GeoDataset(tiff_image1)
tiff_geo2 = GeoDataset(tiff_image2)

# Setup and redefine all 0 values as NaNs
arr_image1 = tiff_geo1.read_array(1)
arr_image1[arr_image1 == 0] = np.NaN

arr_image2 = tiff_geo2.read_array(1)
arr_image2[arr_image2 == 0] = np.NaN

# Image Summary Box

This information is based on the current images you are using. All information is pulled from the images themselves.

In [ ]:
image_summary(tiff_geo1, tiff_geo2)

In [ ]:
show_initial_coregistration(arr_image1, arr_image2, alpha = .5)

In [ ]:
# Generate a dataframe of points associated with a grid where each point
# in the grid is seperated by
df = generate_point_grid(tiff_geo1, tiff_geo2, arr_image1, arr_image2, grid_size)

In [ ]:
comp_df, H, mask = compute_offsets(df, tiff_geo1, tiff_geo2, template_size, search_size, corr_threshold)

In [ ]:
H

# CDF Graphs

This next cell calcullates CDF graphs for the magnitude of x-offets and y-offsets, it also displays the CDF line for x-offset and y-offset individually.

In [ ]:
calculate_cdf_graphs(comp_df, df_mask=True)

# Statistical Data

The three cells bellow are all focused on the statistics of the computation.

## Units

__Xoff__: All pixel values, displaying stats on the shift in the x direction <br>
__Yoff__: All pixel values, displaying stats on the shift in the y direction <br>
__Corr__: All as percentages, displaying stats on the correlation between point comparisons in the current calculation

This goes for both the table bellow and the three box plots bellow.

In [ ]:
stats_file_name = 'stats.csv'

# Add units / calculates stats
stats = comp_df[mask][['xoff', 'yoff', 'mag', 'corr']].describe([.25, .5, .75, .99])

# # Calculates rms and adds it to the dataframe
mean_squared = np.square(stats.loc['mean'])
std_squared = np.square(stats.loc['std'])
rms = np.sqrt(mean_squared + std_squared)
stats.loc['rms'] = rms
stats = stats.sort_index()

# Converts to CSV using the stats_file_name variable
stats.to_csv(stats_file_name, index=False)

stats

# Cumulative Statistics CSV

Running the next cell will produce a CSV of the cumulative statistics of whatever column __('xoff, 'yoff', 'mag')__ you choose.

In [ ]:
cum_df = calculate_cumulative_statistics_df(comp_df)

cum_df

# Run this cell for a CSV of the statistics from above

__stats_file_name__: This can be used for a regular file_name and will convert to a CSV in the same directory or it can be              used to specify a filepath to write the CSV to.

In [ ]:
# file_name for CSV 
stats_file_name = 'stats.csv'

# Creates the combined stats dataframe
new_cum_df = pd.concat([stats, cum_df], axis=1)

# Converts the dataframe to CSV
new_cum_df.to_csv(stats_file_name, encoding='utf-8')

In [ ]:
# Add descriptions/units
plot = comp_df[mask][['xoff', 'yoff', 'mag']].plot(kind='box', figsize=(10, 10))
plot.set_ylabel('Pixel Offset (pixels)', fontsize=18)
plt.show()

In [ ]:
plot = comp_df['corr'][mask].plot(kind='box', figsize=(10, 10))
plot.set_ylabel('Percentage', fontsize=18)
plt.show()

# Quiver Plot Display

Uses the comp_df's x, and y offsets to generate the quiver arrows. While they seem exaggerated they size of a quiver is relative to it's magnitude. Where scale uses the magnitude to draw itself.

In [ ]:
display_quiver(comp_df[mask], arr_image2, scale = 5)

# Quiver Information to CSV
Run the next cell to convert the comp_df to CSV, allowing you to load it into another GIS program.

In [ ]:
# file_name for CSV 
quiver_file_name = 'name_of_quiver.csv'

# Converts above cell to CSV
comp_df.to_csv(quiver_file_name, encoding='utf-8')

# Single Point Display

The first value is the point on the image from left to right, bottom to top.<br>That is, the leftmost, bottom point is 0, the next to the right is 1, etc.

The second value is the size of the area to display in pixels as a square.

In [ ]:
examine_point(0, 100, comp_df[mask], arr_image1, arr_image2)

# Homography Application

Uses the homography generated by the initial compute offsets function to realign the initial images.
<br>This is only as accurate as the homography that was generated and is more or less a sanity check.

In [ ]:
dem_geo1 = GeoDataset(ctx_dem1)
dem_geo2 = GeoDataset(ctx_dem2)

# Setup and redefine all 0 values as NaNs
arr_image1 = dem_geo1.read_array(1)
arr_image1[np.isclose(arr_image1, -3.4028227e+38)] = np.NaN

arr_image2 = dem_geo2.read_array(1)
arr_image2[np.isclose(arr_image2, -3.4028227e+38)] = np.NaN

In [ ]:
new_image = apply_homography(comp_df[mask], arr_image2, H)

In [ ]:
show_coregistration(new_image, arr_image1, alpha = .5)

In [ ]:
plt.figure(figsize=(30, 30))
diff = new_image - arr_image1
plt.imshow(diff)

In [ ]:
diff = abs(diff)
series = pd.Series(data = diff.flatten())
series.dropna()
series.describe()